# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
!pip install Pillow
!pip install scipy==1.1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
project_folder='./Project_data'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from zipfile import ZipFile
dir = "/content/drive/My Drive/Project_data.zip"
with ZipFile(dir) as z:
  z.extractall()

In [ ]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2
import abc

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D,Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications import mobilenet

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
class ModelBuilder(metaclass= abc.ABCMeta):
  def initialize_path(self,project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path =  project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        self.curr_dt_time = datetime.datetime.now()
  def initialize_image_properties(self,image_height,image_width,bigger_picture_cropping_ratio_hight=0,bigger_picture_cropping_ratio_width=0,smaller_picture_cropping_ratio_hight=0,smaller_picture_cropping_ratio_width=0):
        self.image_height=image_height
        self.image_width=image_width
        self.channels=3
        self.num_classes=5
        self.total_frames=30
        self.bigger_picture_cropping_ratio_hight=bigger_picture_cropping_ratio_hight
        self.bigger_picture_cropping_ratio_width=bigger_picture_cropping_ratio_width
        self.smaller_picture_cropping_ratio_hight=smaller_picture_cropping_ratio_hight
        self.smaller_picture_cropping_ratio_width=smaller_picture_cropping_ratio_width
  def initialize_hyperparams(self,frames_to_sample,batch_size,num_epochs):
        self.frames_to_sample=frames_to_sample
        self.batch_size=batch_size
        self.num_epochs=num_epochs
  def train_model(self, model):
        train_generator = self.generator(self.train_path, self.train_doc)
        val_generator = self.generator(self.val_path, self.val_doc)
        model_name = 'model_init' + '_' + str(self.curr_dt_time).replace(' ','').replace(':','_') + '/'
        if not os.path.exists(model_name):
          os.mkdir(model_name)
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
        callbacks_list = [checkpoint, LR]
        if (self.num_train_sequences%self.batch_size) == 0:
          steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
          steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1
        if (self.num_val_sequences%self.batch_size) == 0:
          validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
          validation_steps = (self.num_val_sequences//self.batch_size) + 1
        model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                                callbacks=callbacks_list, validation_data=val_generator, 
                                validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
  def generator(self,source_path, folder_list):
        print( 'Source path = ', source_path, '; batch size =', self.batch_size)
        if (self.total_frames-self.frames_to_sample)%2==0:
          img_idx = [x for x in range((self.total_frames-self.frames_to_sample)//2,self.total_frames-(self.total_frames-self.frames_to_sample)//2)]
        else:
          img_idx = [x for x in range((self.total_frames-self.frames_to_sample)//2+1,self.total_frames-(self.total_frames-self.frames_to_sample)//2)] 
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//self.batch_size
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data = np.zeros((self.batch_size,self.frames_to_sample,self.image_height,self.image_width,self.channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((self.batch_size,self.num_classes)) # batch_labels is the one hot representation of the output
                for folder in range(self.batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*self.batch_size)].split(';')[0]) # read all the images in the folder
                    for idx, item in enumerate(img_idx):  #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*self.batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                        
                        #crop the images and resize them. Note that the images are of 2 different shape
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        if image.shape[0] == image.shape[1]:
                          image=image[self.bigger_picture_cropping_ratio_hight:image.shape[0],self.bigger_picture_cropping_ratio_width:image.shape[1]]
                        else:
                          image=image[self.smaller_picture_cropping_ratio_hight//3:image.shape[0],self.smaller_picture_cropping_ratio_width//2:image.shape[1]]
                        image=imresize(image,[self.image_height,self.image_width])
                        batch_data[folder,idx,:,:,0] = (image[:, : , 0] - np.min(image[:, : , 0]))/(np.max(image[:, : , 0]) - np.min(image[:, : , 0]))
                        batch_data[folder,idx,:,:,1] = (image[:, : , 1] - np.min(image[:, : , 1]))/(np.max(image[:, : , 1]) - np.min(image[:, : , 1]))
                        batch_data[folder,idx,:,:,2] = (image[:, : , 2] - np.min(image[:, : , 2]))/(np.max(image[:, : , 2]) - np.min(image[:, : , 2]))
                        
                    batch_labels[folder, int(t[folder + (batch*self.batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            
            
            # write the code for the remaining data points which are left after full batches
            if (len(folder_list) != self.batch_size*num_batches):
                batch_size = len(folder_list) - (self.batch_size*num_batches)
                batch_data = np.zeros((batch_size,self.frames_to_sample,self.image_height,self.image_width,self.channels)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,self.num_classes)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx, item in enumerate(img_idx):  #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                        
                        #crop the images and resize them. Note that the images are of 2 different shape
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        if image.shape[0] == image.shape[1]:
                          image=image[self.bigger_picture_cropping_ratio_hight:image.shape[0],self.bigger_picture_cropping_ratio_width:image.shape[1]]
                        else:
                          image=image[self.smaller_picture_cropping_ratio_hight//3:image.shape[0],self.smaller_picture_cropping_ratio_width//2:image.shape[1]]
                        image=imresize(image,[self.image_height,self.image_width])
                        
                        batch_data[folder,idx,:,:,0] = (image[:, : , 0] - np.min(image[:, : , 0]))/(np.max(image[:, : , 0]) - np.min(image[:, : , 0]))
                        batch_data[folder,idx,:,:,1] = (image[:, : , 1] - np.min(image[:, : , 1]))/(np.max(image[:, : , 1]) - np.min(image[:, : , 1]))
                        batch_data[folder,idx,:,:,2] = (image[:, : , 2] - np.min(image[:, : , 2]))/(np.max(image[:, : , 2]) - np.min(image[:, : , 2]))
                        
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
  
  @abc.abstractmethod
  def define_model(self):
        pass          

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

**RNN CNN transfer Learning Final Model**

In [ ]:
mobilenet_transfer = mobilenet.MobileNet(weights='imagenet', include_top=False)

class RNNCNN_TL(ModelBuilder):
    
    def define_model(self,gru_cells=64,dense_neurons=64,dropout=0.25):
        
        model = Sequential()
        model.add(TimeDistributed(mobilenet_transfer,input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        
        
        for layer in model.layers:
            layer.trainable = False
        
        
        model.add(TimeDistributed(BatchNormalization()))
        model.add(TimeDistributed(MaxPooling2D((2, 2))))
        model.add(TimeDistributed(Flatten()))

        model.add(GRU(gru_cells))
        model.add(Dropout(dropout))
        
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(Dropout(dropout))
        
        model.add(Dense(self.num_classes, activation='softmax'))
        
        
        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        print (model.summary())
        return model

**Running the Final model**

In [ ]:
conv_3d1=RNNCNN_TL()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=120,image_width=160,bigger_picture_cropping_ratio_hight=50,smaller_picture_cropping_ratio_hight=20,smaller_picture_cropping_ratio_width=20)
conv_3d1.initialize_hyperparams(frames_to_sample=20,batch_size=60,num_epochs=3)
conv_3d1_model=conv_3d1.define_model(gru_cells=128,dense_neurons=128,dropout=0.25)
conv_3d1.train_model(conv_3d1_model)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_8 (TimeDist (None, 20, 3, 5, 1024)    3228864   
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 3, 5, 1024)    4096      
_________________________________________________________________
time_distributed_10 (TimeDis (None, 20, 1, 2, 1024)    0         
_________________________________________________________________
time_distributed_11 (TimeDis (None, 20, 2048)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               836352    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:60: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


Epoch 1/3
10/12 [========================>.....] - ETA: 9s - loss: 1.7184 - categorical_accuracy: 0.2738 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:85: DeprecationWarning:     `imread` is deprecated!
    `imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


12/12 [==============================] - 67s 6s/step - loss: 1.6622 - categorical_accuracy: 0.2958 - val_loss: 1.2553 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to model_init_2021-02-0800_33_32.154346/model-00001-1.47403-0.36953-1.25528-0.45000.h5
Epoch 2/3
12/12 [==============================] - 68s 6s/step - loss: 0.7725 - categorical_accuracy: 0.7512 - val_loss: 0.7950 - val_categorical_accuracy: 0.7400

Epoch 00002: saving model to model_init_2021-02-0800_33_32.154346/model-00002-0.72096-0.76320-0.79500-0.74000.h5
Epoch 3/3
12/12 [==============================] - 73s 7s/step - loss: 0.4263 - categorical_accuracy: 0.8746 - val_loss: 0.5466 - val_categorical_accuracy: 0.8500

Epoch 00003: saving model to model_init_2021-02-0800_33_32.154346/model-00003-0.41107-0.87632-0.54663-0.85000.h5
